In [1]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [2]:
from zkstats.core import create_dummy, verifier_define_calculation, prover_gen_settings, setup, prover_gen_proof, verifier_verify, generate_data_commitment

In [3]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')
data_commitment_path = os.path.join('shared/data_commitment.json')
precal_witness_path = os.path.join('shared/precal_witness_arr.json')

=======================  ZK-STATS FLOW =======================

In [4]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [5]:
# note scale = 2, or 3 makes it more precise, but too big.
scales = [1]
selected_columns = ['x', 'y']
generate_data_commitment(data_path, scales, data_commitment_path)

In [6]:
# Verifier/ data consumer side:
from zkstats.computation import computation_to_model, State

def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    # can AND condition! (same for OR as well)
    filter = torch.logical_and((x>50), (y>300))
    filtered_x = s.where(filter, x)
    filtered_y = s.where(filter, y)
    return s.covariance(filtered_x, filtered_y)

error = 0.01

_, prover_model = computation_to_model(computation, precal_witness_path, True, selected_columns, error)
prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model, prover_model_path, scales, "resources", settings_path)



/Users/jernkun/Desktop/zk-stats-lib/zkstats/computation.py:254: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  is_precise_aggregated = torch.tensor(1.0)
/Users/jernkun/Desktop/zk-stats-lib/zkstats/ops.py:34: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  return torch.tensor(True)
/Users/jernkun/Library/Caches/pypoetry/virtualenvs/zkstats-OJpceffF-py3.11/lib/python3.11/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is depre

==== Generate & Calibrate Setting ====




 <------------- Numerical Fidelity Report (input_scale: 1, param_scale: 1, scale_input_multiplier: 10) ------------->

+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| mean_error | median_error | max_error  | min_error | mean_abs_error | median_abs_error | max_abs_error | min_abs_error | mean_squared_error | mean_percent_error | mean_abs_percent_error |
+------------+--------------+------------+-----------+----------------+------------------+---------------+---------------+--------------------+--------------------+------------------------+
| 0.08456421 | 0.16912842   | 0.16912842 | 0         | 0.08456421     | 0.16912842       | 0.16912842    | 0             | 0.014302211        | 0.00025573664      | 0.00025573664          |
+------------+--------------+------------+-----------+----------------+------------------+---------------+-------------

scale:  [1]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":1,"param_scale":1,"scale_rebase_multiplier":10,"lookup_range":[-1348,896],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Fixed","div_rebasing":false,"rebase_frac_zero_constants":false,"check_mode":"UNSAFE"},"num_rows":13120,"total_assignments":7463,"total_const_size":2836,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,1],"model_input_scales":[1,1],"module_sizes":{"kzg":[],"poseidon":[13120,[2]]},"required_lookups":[{"GreaterThan":{"a":0.0}},"Abs"],"required_range_checks":[],"check_mode":"UNSAFE","version":"9.1.0","num_blinding_factors":null,"timestamp":1715153321207}


In [7]:
_, verifier_model = computation_to_model(computation, precal_witness_path, False, selected_columns, error)
verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path,verifier_model, verifier_model_path)

In [8]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====
Time setup: 1.7946860790252686 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 330.5
==== Generating Proof ====
proof:  {'instances': [['b8c4693b3424042141e265e5179743499536c8b9f1ddc7d4d4fc847c1c09df04', '7d055c183092753c601560dfbfa0b446f5417438227d99435c476a0659de3d14', '0100000000000000000000000000000000000000000000000000000000000000', '9502000000000000000000000000000000000000000000000000000000000000']], 'proof': '0x1edf52114f2977d45ceb9607f10c0f560e60d620b32e999c52fc48871a72cfb8062384f8c8aba458a120b4dda578f8bc768b22055ae24348fa889f88d863cdfd0323e98fbe3c3ee0e94b0266ab0e18d038abbc8141f683dd11bf884694708ca017945af5130d81b8554704c61a8a1056b6585c2e7df6b9b54b79f55bca893dad0bc0bdb9568af729ebfa625f7d13215d041abb8d916dfe65660bee12903b8a4805e777e871aa4669fbdc35f6cd3150500dba147a63cb45877a14827fb7a2a43f1a74bcf04a79a9ed5986b1e4d45fea621cffd0656f2ca0083b2d2398e9724f5b2f2d206687c3e142d724ad6cbf7b85c69b32bbf76965278289f36259183d00dc02d4570473

In [9]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, data_commitment_path)
print("Verifier gets result:", res)

Verifier gets result: [330.5]
